In [14]:
import psycopg2 as ps
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

In [2]:
# Load environment variables
load_dotenv()
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_host = os.getenv('DB_HOST')
db_name = os.getenv('DB_NAME')

### Connect to the database

In [15]:
conn = ps.connect(
    host=db_host,
    dbname=db_name,
    user=db_user,
    password=db_password
)

### Creating a SQLAlchemy Engine to be used ETL

In [16]:
engine = create_engine(f"postgresql+psycopg2://{db_user}:{db_password}@{db_host}/{db_name}")
connection = engine.connect()

In [4]:
# Load sql extension to allow me to run sql queries in cells
%load_ext sql

In [5]:
# Configuration for using sql magic
%config SqlMagic.autocommit=False #disables committing to a db automatically
%config SqlMagic.displaycon = False #disables info from db to be displayed
%config SqlMagic.feedback = False #disables info on affected rows to be displayed
%config SqlMagic.autopandas = True #allow us to convert sql query to pandas df automatically
#set up connection to db
%sql postgresql://{db_user}:{db_password}@{db_host}/{db_name} 

In [6]:
# Check what the bookings table look like
%sql SELECT * FROM bookings LIMIT 5;

,bookid,facid,memid,starttime,slots
0,0,3,1,2012-07-03 11:00:00,2
1,1,4,1,2012-07-03 08:00:00,2
2,2,6,0,2012-07-03 18:00:00,2
3,3,7,1,2012-07-03 19:00:00,2
4,4,8,1,2012-07-03 10:00:00,1


In [7]:
%%sql
SELECT memid, COUNT(*) as total_bookings
FROM bookings
GROUP BY memid
ORDER BY total_bookings DESC
LIMIT 5;

,memid,total_bookings
0,0,883
1,3,408
2,1,261
3,2,210
4,8,188


This looks good so let's write this again and save this to a dataframe

In [9]:
query = """
SELECT memid, COUNT(*) as total_bookings
FROM bookings
GROUP BY memid
ORDER BY total_bookings DESC
LIMIT 5;
"""

In [19]:
total_bookings_per_member = pd.read_sql_query(query, con=connection)
total_bookings_per_member

,memid,total_bookings
0,0,883
1,3,408
2,1,261
3,2,210
4,8,188


This dataframe looks good, let's create a table in psql locally to load this dataframe to

In [20]:
def execute_query_postgresql(conn_string, query):
    with ps.connect(conn_string) as conn:
        with conn.cursor() as cur:
            cur.execute(query)
            conn.commit()

# Define the connection string for your analytical database
etl_bites_conn_string = "host='localhost' port='5432' dbname='etl_bites' user='denisechan'"

# SQL query to create a new table for storing total booking duration per facility
create_top_5_members_bookings = '''
CREATE TABLE top_5_members_bookings (
    MemberID INTEGER NOT NULL,
    TotalBookings INTEGER NOT NULL
);
'''

# Execute the query to create the table
execute_query_postgresql(etl_bites_conn_string, create_top_5_members_bookings)

Load the dataframe to the psql table locally

In [21]:
def insert_data(conn_string, table_name, data, columns):
    with ps.connect(conn_string) as conn:
        with conn.cursor() as cur:
            for row in data.itertuples(index=False):
                insert_query = f"INSERT INTO {table_name} ({', '.join(columns)}) VALUES ({', '.join(['%s'] * len(columns))});"
                cur.execute(insert_query, row)
            conn.commit()

# Insert the transformed data into the analytical database
insert_data(etl_bites_conn_string, 'top_5_members_bookings', total_bookings_per_member, ['MemberID', 'TotalBookings'])